# TP2: Fine-Tuning Simplified AlphaFold

**Day 3 - AI for Sciences Winter School**

**Instructor:** Raphael Cousin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/raphaelcousin/ai_for_sciences/blob/main/day3/tp2.ipynb)

## Topics
- Protein structure prediction
- Fine-tuning techniques for AlphaFold